In [1]:
%%capture
%run 01_hash.ipynb
%run 02_transaction.ipynb

In [2]:
import subprocess as sub

## 99_contract

Our goal is to be able to run a small piece of code in a tx, just like Ethereum. This piece of code is also called a smart contract (sc). The sc is like another user, but in addition to the balance it also has storage for variables and the sc itself. 

The smart contract should be able to access the current tx and block information.

## TODO:

- Add some sort of typing system. r:int = 10
- Inject block info

In [3]:
code = """
def init():
    r = 0
    s = 2

r += 5
time = _tx["time"] 
"""

In [4]:
SC = 'sc.py'

PRINT_VARS = ['for n,v in globals().copy().items():',
              '    if n[0] not in [\'_\', \'[\']:',
              '        print(n,v)']

PRINT_VARS = ['local = set(locals())',
              'for n in local:',
              '    if not n.startswith(\'_\'):',
              '        print(n + \'__\', eval(n))']

In [5]:
MEMORY = {}

In [6]:
def inject(tx, code): 
    tx = tx.json().replace('true', 'True')
    return '_tx = '+tx+'\n'+code

In [7]:
def deployment(code):
    init = []
    lines = code.split('\n')
    for i,l in enumerate(lines):
        if 'init' in l: 
            for j in range(i+1, len(lines)):
                if lines[j][:1] == ' ': init.append(lines[j].replace(' ', ''))
                else                  : break
    init.extend(PRINT_VARS)
    return '\n'.join(l for l in init)

In [8]:
def save(code):
    with open(SC, 'w+') as f: f.write(code)

In [9]:
def run():
    r = sub.check_output(f'python3 {SC}', shell=True)
    return r.decode().split('\n')[:-1]

In [10]:
def store(r):
    for var in r: MEMORY[var.split('__')[0]]= var.split('__')[1][1:]

In [11]:
def deploy(code, tx):
    dc = deployment(code)
    ic = inject(tx, dc)
    save(ic)
    r  = run()
    store(r)

deploy(code, tx1); MEMORY

{'s': '2', 'r': '0'}

In [12]:
def execution(code):
    c = []
    for k,v in MEMORY.items(): c.append(f'{k} = {v}')
    lines = code.split('\n')
    in_init = False
    for l in lines:
        if 'init' in l: in_init = True; continue
        if in_init:
            if l[:1] == ' ': continue
            else           : in_init = False
        else: c.append(l)
    c.extend(PRINT_VARS)
    return '\n'.join(l for l in c)

In [13]:
def exe(code, tx):
    ic = inject(tx, code)
    ec = execution(ic)
    save(ec)
    r  = run()
    store(r)
    
exe(code, tx1)

In [14]:
MEMORY

{'s': '2', 'r': '5', 'time': 'Mon Apr 12 00:03:38 2021'}